In [1]:
from scipy.optimize import root
import numpy as np
import pickle

In [2]:
with open('mf1.pkl', 'rb') as f:
    ss_params,model_params,ss_shock = pickle.load(f)


In [3]:
def assign_params(ss_params):
    a_0 = ss_params['a_0']
    a_1 = ss_params['a_1']
    a_2 = ss_params['a_2']
    alf = ss_params['alf']
    delt = ss_params['delt']
    om_0 = ss_params['om_0']
    om_1 = ss_params['om_1']
    om_2 = ss_params['om_2']
    eps = ss_params['eps']
    tht = ss_params['tht']
    bet = ss_params['bet']
    psi_k = ss_params['psi_k']
    psi_n = ss_params['psi_n']
    return eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2


In [4]:
ss_params

{'eps': 0.5,
 'tht': 1,
 'alf': 0.3,
 'bet': 0.9,
 'om_0': 0.3,
 'om_1': 0.3,
 'om_2': 0.2,
 'delt': 0.05,
 'psi_n': 1,
 'psi_k': 1,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [5]:
def equations(vars, params):
	n_0,n_1,n_2,s_0,s_1,s_2,k,c_0,c_1,c_2,i = vars
	eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
 
	
	#ph=(om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))

	v_0=om_0*c_0**(-1/eps)
	v_1=om_1*c_1**(-1/eps)
	v_2=om_2*c_2**(-1/eps)

	fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0)**alf*k**alf
	fn_1=np.exp(a_1)*(1-alf)*(s_1/n_1)**alf*k**alf
	fn_2=np.exp(a_2)*(1-alf)*(s_2/n_2)**alf*k**alf
 
	fs_0=np.exp(a_0)*alf*(n_0/s_0)**(1-alf)*k**alf
	fs_1=np.exp(a_1)*alf*(n_1/s_1)**(1-alf)*k**alf
	fs_2=np.exp(a_2)*alf*(n_2/s_2)**(1-alf)*k**alf
  
	x_10=fn_0
	x_11=fn_1
	x_12=fn_2

	x_20=0
	x_21=0
	x_22=0
 
	z_1=1
	z_2=1-delt
 
	eq1=n_0+n_1+n_2-1
	eq2=s_0+s_1+s_2-1
	eq3=v_1*fn_1-v_0*fn_0
	eq4=v_2*fn_2-v_0*fn_0
	eq5=v_1*fs_1-v_0*fs_0
	eq6=v_2*fs_2-v_0*fs_0
	eq7=bet*(fs_0/k+z_2)-1
	eq8=(1-delt)*k+i-k
	eq9=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf
	eq10=c_1-np.exp(a_1)*n_1**(1-alf)*s_1**alf*k**alf
	eq11=c_2-np.exp(a_2)*n_2**(1-alf)*s_2**alf*k**alf
	
	return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11]


In [6]:
initial_guess = [1,1,.3,.3,.3,.3,3,1,1,1,.1]
def solve_equations(params):
    
    sol = root(equations, initial_guess, args=(ss_params,))
    
    # Extract the solution vector
    n_0,n_1,n_2,s_0,s_1,s_2,k,c_0,c_1,c_2,i = sol.x
    #eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
    a_0=0
    a_1=0
    a_2=0
    ss_dict = {
    #'n_0': n_0,
    'n_1': n_1,
    'n_2': n_2,
    #'s_0': s_0,
    's_1': s_1,
    's_2': s_2,    
    'k': k,
    'c_0': c_0,
    'c_1': c_1,
    'c_2': c_2,
    'i': i,
    'a_0': a_0,
    'a_1': a_1,
    'a_2': a_2
    }
    return sol, ss_dict
    

#Call the wrapper function
sol, ss_dict=solve_equations(ss_params)
print(sol)
ss_dict


 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00 -3.857e-10 -1.869e-10 -2.005e-10
            1.105e-10 -2.651e-11  4.441e-16 -1.765e-11 -9.579e-12
           -1.087e-11]
       x: [ 4.151e-01  3.220e-01  2.629e-01  4.151e-01  3.220e-01
            2.629e-01  2.431e+00  4.203e-01  4.203e-01  3.432e-01
            1.215e-01]
    nfev: 74
    fjac: [[-4.230e-01  3.645e-10 ...  2.675e-03  1.825e-03]
           [-5.739e-01 -8.530e-11 ...  4.215e-01  1.117e-03]
           ...
           [-3.385e-01 -7.573e-02 ... -3.184e-01 -3.133e-01]
           [ 2.571e-02 -7.316e-02 ... -6.584e-02 -3.890e-02]]
       r: [-2.364e+00  6.311e-01 ...  2.263e-01  6.550e-01]
     qtf: [-5.705e-10 -7.434e-11  6.695e-10 -3.283e-10  2.743e-10
            1.126e-10 -2.358e-10 -1.047e-09  1.099e-09 -6.624e-11
           -1.276e-10]


C:\Users\blankenw\AppData\Local\Temp\ipykernel_8376\4277247800.py:13: RuntimeWarning: invalid value encountered in scalar power
  fn_1=np.exp(a_1)*(1-alf)*(s_1/n_1)**alf*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_8376\4277247800.py:17: RuntimeWarning: invalid value encountered in scalar power
  fs_1=np.exp(a_1)*alf*(n_1/s_1)**(1-alf)*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_8376\4277247800.py:40: RuntimeWarning: invalid value encountered in scalar power
  eq10=c_1-np.exp(a_1)*n_1**(1-alf)*s_1**alf*k**alf


{'n_1': 0.3219945509205003,
 'n_2': 0.2629074499028316,
 's_1': 0.3219945509563271,
 's_2': 0.2629074498427949,
 'k': 2.43057054964539,
 'c_0': 0.42030154999653896,
 'c_1': 0.42030155005211545,
 'c_2': 0.34317477853670936,
 'i': 0.12152852748226979,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [7]:
ss_dict

{'n_1': 0.3219945509205003,
 'n_2': 0.2629074499028316,
 's_1': 0.3219945509563271,
 's_2': 0.2629074498427949,
 'k': 2.43057054964539,
 'c_0': 0.42030154999653896,
 'c_1': 0.42030155005211545,
 'c_2': 0.34317477853670936,
 'i': 0.12152852748226979,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [8]:
# Call the wrapper function
#sol, ss_dict = solve_equations(ss_params)

In [9]:
# Replace values with their logarithms except for the key 'i'
for key in ss_dict:
	if  key not in ['i', 'a_0','a_1','a_2']:
		ss_dict[key] = np.log(ss_dict[key])



In [10]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('var \n')
        for key, value in ss_dict.items():
            f.write(f'{key}\n')
        f.write(f';\n\n\n')
        f.write('varexo \n')
        for key, value in ss_shock.items():
            f.write(f'{key}_shock\n')
        f.write(f';\n\n\n')
# Call the function
write_dict_to_file(ss_dict, 't_vars.m')

In [11]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('initval;\n')
        for key, value in ss_dict.items():
            f.write(f'{key}={value};\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_ss.m')

In [12]:
ss_dict

{'n_1': -1.1332206561880598,
 'n_2': -1.3359532102463847,
 's_1': -1.1332206560767943,
 's_2': -1.3359532104747414,
 'k': 0.888126023885391,
 'c_0': -0.8667828491440832,
 'c_1': -0.8667828490118531,
 'c_2': -1.069515403180939,
 'i': 0.12152852748226979,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [13]:
def write_dict_to_file(params, filename):
    
    with open(filename, 'w') as f:
        f.write('parameters ')
        f.write(','.join(f'{key}' for key in params))
        f.write(';\n\n')
        for key, value in params.items():
            f.write(f'{key}={value};\n')

# Call the function
write_dict_to_file(model_params, 't_parameters.m')

In [14]:
def write_dict_to_file(ss_dict, filename):
    #variance_list = list(shock_params.values())[:3]
    with open(filename, 'w') as f:
        f.write('shocks;\n')
        i=0
        for key, value in ss_shock.items():
            f.write(f'var {key}_shock=var_{key};\n')
            i+=1
        f.write(f'\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_shocks.m')

In [15]:
with open('mf2.pkl', 'wb') as f:
    pickle.dump((ss_dict), f)